# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,10.27,82,88,4.50,CL,1707283614
1,1,khanu woralaksaburi,16.0617,99.8606,34.41,19,0,0.73,TH,1707283614
2,2,bethel,41.3712,-73.4140,0.02,74,0,0.00,US,1707283615
3,3,henderson,36.0397,-114.9819,9.43,63,100,6.69,US,1707283615
4,4,kourou,5.1552,-52.6478,26.03,92,18,3.36,GF,1707283615


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [73]:
%%capture --no-display

# Configure the map plot
cities_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['City'],
    alpha = 0.5
)

# Display the map
cities_humidity



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [57]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_cities = city_data_df.loc[(city_data_df['Max Temp'] < 22) & (city_data_df['Max Temp'] > 10) & (city_data_df['Humidity'] < 45)]

# Drop any rows with null values
cleaned_perfect_cities = perfect_cities.dropna(how='any')

# Display sample data
cleaned_perfect_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,mallammaduri,12.5650,9.9573,16.53,22,83,4.97,NG,1707283636
56,56,nouadhibou,20.9310,-17.0347,19.99,34,16,3.60,MR,1707283637
99,99,birendranagar,28.6019,81.6339,18.35,26,73,1.70,NP,1707283651
119,119,hun,29.1268,15.9477,11.45,39,0,2.19,LY,1707283658
137,137,geneina,13.4500,22.4500,14.37,26,16,7.60,SD,1707283665


### Step 3: Create a new DataFrame called `hotel_df`.

In [58]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_perfect_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
53,mallammaduri,NG,12.5650,9.9573,22,
56,nouadhibou,MR,20.9310,-17.0347,34,
99,birendranagar,NP,28.6019,81.6339,26,
119,hun,LY,29.1268,15.9477,39,
137,geneina,SD,13.4500,22.4500,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1

params = {
    'categories':categories,
    'apiKey':geoapify_key,
    limit:limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = cleaned_perfect_cities.loc[index, 'Lat']
    longitude = cleaned_perfect_cities.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mallammaduri - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
birendranagar - nearest hotel: Aacharya
hun - nearest hotel: فندق الهروج
geneina - nearest hotel: Princess
nasir - nearest hotel: منزل طارق
masterton - nearest hotel: Copthorne Hotel Solway Park
mount gambier - nearest hotel: The Old Mount Gambier Gaol
carnot - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
al muwayh - nearest hotel: فندق ومطعم الموية
new norfolk - nearest hotel: Woodbridge on the Derwent
faya-largeau - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
egbe - nearest hotel: No hotel found
colac - nearest hotel: No hotel found
al mutayrifi - nearest hotel: No hotel found
al fqih ben calah - nearest hotel: Hotel Oum Elfadl
phalodi - nearest hotel: LAL NIWAS
nguigmi - nearest hotel: Guest PAM
barah - nearest hotel: No hotel found
buta - nearest hotel: Hôtel Rubi
al hazm - nearest hotel: No hote

,City,Country,Lat,Lng,Humidity,Hotel Name
53,mallammaduri,NG,12.5650,9.9573,22,No hotel found
56,nouadhibou,MR,20.9310,-17.0347,34,Hotel valencia
99,birendranagar,NP,28.6019,81.6339,26,Aacharya
119,hun,LY,29.1268,15.9477,39,فندق الهروج
137,geneina,SD,13.4500,22.4500,26,Princess
155,nasir,SD,15.5894,32.5634,18,منزل طارق
177,masterton,NZ,-40.9597,175.6575,20,Copthorne Hotel Solway Park
185,mount gambier,AU,-37.8333,140.7667,42,The Old Mount Gambier Gaol
196,carnot,CF,4.9409,15.8665,37,No hotel found
222,i-n-salah,DZ,27.1935,2.4607,27,باجودة


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country'],
    alpha = 0.5
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)